# Q5.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
Imgs="../input/devanagari-character-set/"
for i in os.listdir(Imgs):
    print(i)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Segregating Images and lokooking into the data

In [2]:
import pandas as pd

Checking For data.csv file

In [3]:
train = pd.read_csv(Imgs+"data.csv")

In [4]:
train=pd.DataFrame(train)

In [5]:
print(train.isnull().sum())

In [6]:
train.dtypes

In [7]:
#Extracting Last column from the table
Column_name=train["character"].to_numpy()
del train["character"]

In [8]:
Column_name

In [9]:
train.to_numpy(dtype='int')

In [10]:
train=np.array(train).astype('int')

Visualising Images

In [11]:
train.shape

In [12]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
dict={}

count=0
#range(start,stop,step)
for i in range(0,92000,2000):
    imarray=np.array(train[i].reshape(32,32))
    dict.update({count:imarray}) 
    count=count+1
    if count >= 92000:
        break
       
figure, ax = plt.subplots(nrows=8,ncols=6,figsize=(10,10) )
for ind,title in enumerate(dict):
    ax.ravel()[ind].imshow(dict[title])
    ax.ravel()[ind].set_title(title)
    ax.ravel()[ind].set_axis_off()
plt.tight_layout()
plt.show()

Train images and column with One Hot encoding

In [13]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
values = array(Column_name)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(train.astype("float32"),onehot_encoded, test_size=0.1, random_state=42)

In [15]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [16]:
82800/50

In [17]:
9200/50

In [18]:
X_train=(X_train.reshape(82800,32,32))/255
X_test=(X_test.reshape(9200,32,32))/255


In [19]:
X_train.shape

In [20]:
plt.imshow(X_train[44])

Keras Model

In [21]:
from tensorflow.python.keras.layers import Dense,Conv2D,MaxPooling2D,GlobalMaxPooling2D,Flatten,Dropout
from tensorflow.python.keras.models import Sequential
import tensorflow as tf
from keras.layers import Input

In [22]:
model = Sequential(name="my_sequential")
model.add(Conv2D(50,(3,3),activation="relu",kernel_initializer='he_uniform', input_shape=(32,32,1)))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(256,activation="relu",kernel_initializer='he_uniform'))
model.add(Dense(46,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])


In [23]:
model.summary()

Building model is compulsory using Tensorflow libraries

In [24]:
'''Here x_train is grayscale image which is having only one band of color, unlike RGB which has 3 
So, we need to add additional shape here that is 1 reshape the vector.
'''
X_train=X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test=X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

In [25]:
history=model.fit(X_train, y_train, batch_size=50, epochs=10, validation_split=None,validation_data=(X_test,y_test))

In [26]:
"""model.fit(
    x=X_train,
    y=y_train,
    batch_size=None,
    epochs=10,
    verbose="auto",
    callbacks=None,
    validation_split=0.0,
    validation_data=None,
    shuffle=True, 
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=10,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)
"""

In [27]:
model.evaluate(
    x=X_test,
    y=y_test,
    batch_size=None,
    verbose=1,
    sample_weight=None,
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
    return_dict=False,
    #**kwargs
)


In [28]:
#Plot of Model accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()



In [29]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [30]:
"""Model.predict(
    x,
    batch_size=None,
    verbose=0,
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)
"""